In [1]:
pip install featuretools

In [2]:
import numpy as np
import featuretools as ft
import pandas as pd

In [3]:
from featuretools.primitives.standard.transform.datetime.season import date

customers = pd.read_csv('/content/customers.csv',
                        dtype ={'CustomerId': int, 'Name' : str, 'Email' :str , 'SignupDate':str},
                        parse_dates=['SignupDate'])
products = pd.read_csv('/content/products.csv',
                       dtype = {'ProductID': int, 'Name' : str, 'Category' :str , 'Price':float})
orders = pd.read_csv('/content/orders.csv',
                     dtype = {'OrderID': int, 'CustomerID' : int, 'OrderDate' :str , 'ShipDate':str},
                     parse_dates=['OrderDate','ShipDate'] )
orderDetails = pd.read_csv('/content/orderDetails.csv',
                           dtype = {'OrderID': int, 'ProductID' : int, 'Quantity' :int , 'Discount':float})

#remove any nan column
customers = customers.dropna()
products = products.dropna()
orders = orders.dropna()
orderDetails = orderDetails.dropna()

In [4]:
#create entity and entityset
es = ft.EntitySet(id = 'order')

#use existing index
es = es.add_dataframe(dataframe_name= 'products', dataframe = products, index = 'ProductID')
es = es.add_dataframe(dataframe_name= 'customers', dataframe = customers, index = 'CustomerID')
es = es.add_dataframe(dataframe_name= 'orders', dataframe = orders, index = 'OrderID')
#create new unique index
es = es.add_dataframe(dataframe_name= 'orderDetails', dataframe = orderDetails,
    make_index = True, index = 'orderDetails_index')

es

Entityset: order
  DataFrames:
    products [Rows: 10, Columns: 4]
    customers [Rows: 15, Columns: 4]
    orders [Rows: 20, Columns: 4]
    orderDetails [Rows: 20, Columns: 5]
  Relationships:
    No relationships

In [5]:
#create relationship
es = es.add_relationship('customers', 'CustomerID', 'orders', 'CustomerID')
es = es.add_relationship('orders', 'OrderID', 'orderDetails', 'OrderID')
es = es.add_relationship('products','ProductID', 'orderDetails', 'ProductID')

es

Entityset: order
  DataFrames:
    products [Rows: 10, Columns: 4]
    customers [Rows: 15, Columns: 4]
    orders [Rows: 20, Columns: 4]
    orderDetails [Rows: 20, Columns: 5]
  Relationships:
    orders.CustomerID -> customers.CustomerID
    orderDetails.OrderID -> orders.OrderID
    orderDetails.ProductID -> products.ProductID

In [6]:
#Determine primitive
agg_primitives =  ["sum","count", "percent_true", "mode"]
trans_primitives =  ["day", "year", "month", "weekday", "haversine","num_words", "num_characters",
                     "subtract_numeric","add_numeric","multiply_numeric"]

#DFS with specified primitives
feature_matrix, feature_defs = ft.dfs(entityset = es,
    target_dataframe_name = 'orders',
    trans_primitives = trans_primitives,
    agg_primitives=agg_primitives,
    max_depth = 4, n_jobs = -1, verbose = 1)

2023-11-12 05:07:10,067 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Discount * Quantity)> which is already present. This is likely a bug.
2023-11-12 05:07:10,070 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Discount + Quantity)> which is already present. This is likely a bug.
2023-11-12 05:07:10,077 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Discount - Quantity)> which is already present. This is likely a bug.
2023-11-12 05:07:10,078 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Discount)> which is already present. This is likely a bug.
2023-11-12 05:07:10,079 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Quantity)> which is already present. This is likely a bu

/usr/local/lib/python3.10/dist-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['haversine', 'num_characters', 'num_words']
  agg_primitives: ['percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Elapsed: 00:00 | Progress:   0%|          

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40789
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33645'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35435'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40289', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40289
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:43784
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:40591', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40591
INFO:distributed.core:Sta

EntitySet scattered to 2 workers in 6 seconds
Elapsed: 00:02 | Progress:  95%|█████████▌

INFO:distributed.scheduler:Remove client Client-58c6bb22-8119-11ee-b112-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:54780; closing.
INFO:distributed.scheduler:Remove client Client-58c6bb22-8119-11ee-b112-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-58c6bb22-8119-11ee-b112-0242ac1c000c
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:33645'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:35435'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:43794; closing.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:40591', name: 1, status: closing, memory: 1, processing: 0> (stimulus_id='handle-worker-cleanup-1699765644.411336')
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.

Elapsed: 00:04 | Progress: 100%|██████████


In [7]:
feature_matrix

,CustomerID,COUNT(orderDetails),SUM(orderDetails.Discount),SUM(orderDetails.Quantity),DAY(OrderDate),DAY(ShipDate),MONTH(OrderDate),MONTH(ShipDate),WEEKDAY(OrderDate),WEEKDAY(ShipDate),...,customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount * products.Price),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount + Quantity),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount + products.Price),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount - Quantity),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount - products.Price),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.Discount),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.products.Price * Quantity),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.products.Price + Quantity),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.products.Price - Quantity),customers.SUM(orderDetails.products.Price) - SUM(orderDetails.products.Price)
OrderID,,,,,,,,,,,,,,,,,,,,,
301,101,1,0.00,1.0,1,3,2,2,2,4,...,580.0,579.00,80.00,581.00,1080.00,580.00,80.0,79.0,81.0,80.0
302,102,1,0.10,2.0,5,7,2,2,6,1,...,1270.0,1297.90,999.90,1301.90,1599.90,1299.90,700.0,998.0,1002.0,1000.0
303,103,1,0.00,1.0,10,12,2,2,4,6,...,1500.0,1499.00,800.00,1501.00,2200.00,1500.00,800.0,799.0,801.0,800.0
304,104,1,0.20,3.0,15,18,2,2,2,5,...,166.0,166.80,149.80,172.80,189.80,169.80,110.0,147.0,153.0,150.0
305,105,1,0.00,1.0,20,23,2,2,0,3,...,250.0,249.00,200.00,251.00,300.00,250.00,200.0,199.0,201.0,200.0
306,106,1,0.00,2.0,25,28,2,2,5,1,...,80.0,78.00,0.00,82.00,160.00,80.00,-80.0,-2.0,2.0,0.0
307,107,1,0.15,1.0,1,4,3,3,2,5,...,850.0,998.85,-0.15,1000.85,1999.85,999.85,0.0,-1.0,1.0,0.0
308,108,1,0.00,1.0,5,8,3,3,6,2,...,800.0,799.00,0.00,801.00,1600.00,800.00,0.0,-1.0,1.0,0.0
309,109,1,0.10,2.0,10,13,3,3,4,0,...,135.0,147.90,-0.10,151.90,299.90,149.90,-150.0,-2.0,2.0,0.0


In [8]:
feature_defs

[<Feature: CustomerID>,
 <Feature: COUNT(orderDetails)>,
 <Feature: SUM(orderDetails.Discount)>,
 <Feature: SUM(orderDetails.Quantity)>,
 <Feature: DAY(OrderDate)>,
 <Feature: DAY(ShipDate)>,
 <Feature: MONTH(OrderDate)>,
 <Feature: MONTH(ShipDate)>,
 <Feature: WEEKDAY(OrderDate)>,
 <Feature: WEEKDAY(ShipDate)>,
 <Feature: YEAR(OrderDate)>,
 <Feature: YEAR(ShipDate)>,
 <Feature: SUM(orderDetails.Discount * Quantity)>,
 <Feature: SUM(orderDetails.Discount + Quantity)>,
 <Feature: SUM(orderDetails.Discount - Quantity)>,
 <Feature: SUM(orderDetails.products.Price)>,
 <Feature: COUNT(orderDetails) + SUM(orderDetails.Discount)>,
 <Feature: COUNT(orderDetails) + SUM(orderDetails.Quantity)>,
 <Feature: SUM(orderDetails.Discount) + SUM(orderDetails.Quantity)>,
 <Feature: COUNT(orderDetails) * SUM(orderDetails.Discount)>,
 <Feature: COUNT(orderDetails) * SUM(orderDetails.Quantity)>,
 <Feature: SUM(orderDetails.Discount) * SUM(orderDetails.Quantity)>,
 <Feature: COUNT(orderDetails) - SUM(orderDe

In [9]:
feature_matrix.to_csv('/content/result.csv', index=False)